In [1]:
import requests
from bs4 import BeautifulSoup 


In [2]:
#Check whether given website allows web scraping or not, If out 200 then good to go
r=requests.get("https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating")

r.status_code

200

In [15]:
import pandas as pd
import numpy as np

In [16]:
url = 'https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [25]:
movie_name = []
year = []
time = []
ratings = []
metascore = []
gross = []
votes = []
description = []

In [26]:
movie_data = soup.find_all('div', attrs = {'class': 'lister-item mode-advanced'})

In [20]:
#movie_data

In [35]:
for stored in movie_data:
  name = stored.h3.a.text
  movie_name.append(name)

  y = stored.h3.find('span', class_ = 'lister-item-year text-muted unbold').text.replace('(', '').replace(')', '')
  year.append(y)
    
  runtime = stored.p.find('span', class_ = 'runtime').text.replace(' min', '')
  time.append(runtime)
    
  rate = stored.find('div', class_ = 'inline-block ratings-imdb-rating').text.replace('\n', '')
  ratings.append(rate)
    
  meta  = stored.find('span', class_ = 'metascore').text.replace(' ', '') if stored.find('span', class_ = 'metascore') else '#####'
  metascore.append(meta)
  #since, gross and votes have same attributes, so created a common variable and then used indexing
  value = stored.find_all('span', attrs = {'name': 'nv'})
    
  vote = value[0].text
  votes.append(vote)
    
  grosses = value[1].text if len(value) >1 else '*****'
  gross.append(grosses)

  describe = stored.find_all('p', class_ = 'text-muted')
  description_ = describe[1].text.replace('\n', '') if len(describe) >1 else '*****'
  description.append(description_)

In [48]:
#creating dataframe 
a ={'Name of movie': movie_name, 'Year of release': year, 'Watchtime': time, 'Movie Rating': ratings, 'Metascore': metascore, 'Votes': votes, 'Gross collection': gross, 'Description': description}
df = pd.DataFrame.from_dict(a, orient='index')
df = df.transpose()

In [49]:
df

,Name of movie,Year of release,Watchtime,Movie Rating,Metascore,Votes,Gross collection,Description
0,The Shawshank Redemption,1994,142,9.3,80,"2,459,710",$28.34M,Two imprisoned men bond over a number of years...
1,The Godfather,1972,175,9.2,100,"1,701,585",$134.97M,An organized crime dynasty's aging patriarch t...
2,Soorarai Pottru,2020,153,9.1,#####,"89,663",*****,"Nedumaaran Rajangam ""Maara"" sets out to make t..."
3,The Dark Knight,2008,152,9.0,84,"2,415,236",$534.86M,When the menace known as the Joker wreaks havo...
4,The Godfather: Part II,1974,202,9.0,90,"1,181,867",$57.30M,The early life and career of Vito Corleone in ...
...,...,...,...,...,...,...,...,...
301,A Separation,2011,123,8.3,75,"34,985",$47.70M,After his family was brutally murdered and his...
302,Incendies,2010,131,8.3,88,"93,901",*****,A man refuses all assistance from his daughter...
303,Demon Slayer: Mugen Train,2020,117,8.3,#####,"82,957",*****,The family of a left-wing journalist is torn a...
304,The Father,I 2020,97,8.3,None,None,None,None


In [50]:
#change to csv
df.to_csv('movies.csv')

In [51]:
data = pd.read_csv('movies.csv')
data.shape

(306, 9)

In [52]:
#save dataframe to sql database
import sqlite3

In [53]:
conn = sqlite3.connect('TestDB.db')
c = conn.cursor()

In [64]:
c.execute('CREATE TABLE IF NOT EXISTS movies (Name of movie, Year of release, Watchtime, Movie Rating, Metascore, Vote, Gross collection, Description)')
conn.commit()

In [65]:
data.to_sql('movies', conn, if_exists = 'replace', index = False)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:2615: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [66]:
c.execute('''SELECT * FROM movies''')

In [67]:
for row in c.fetchall():
  print(row)

(0, 'The Shawshank Redemption', '1994', 142, 9.3, '80', '2,459,710', '$28.34M', 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.')
(1, 'The Godfather', '1972', 175, 9.2, '100', '1,701,585', '$134.97M', "An organized crime dynasty's aging patriarch transfers control of his clandestine empire to his reluctant son.")
(2, 'Soorarai Pottru', '2020', 153, 9.1, '#####', '89,663', '*****', 'Nedumaaran Rajangam "Maara" sets out to make the common man fly and in the process takes on the world\'s most capital intensive industry and several enemies who stand in his way.')
(3, 'The Dark Knight', '2008', 152, 9.0, '84', '2,415,236', '$534.86M', 'When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice.')
(4, 'The Godfather: Part II', '1974', 202, 9.0, '90', '1,181,867', '$57.30M', 'The early life and

In [82]:
c.execute('''SELECT * FROM movies limit 5''')

In [83]:
for row in c.fetchall():
  print(row)

(0, 'The Shawshank Redemption', '1994', 142, 9.3, '80', '2,459,710', '$28.34M', 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.')
(1, 'The Godfather', '1972', 175, 9.2, '100', '1,701,585', '$134.97M', "An organized crime dynasty's aging patriarch transfers control of his clandestine empire to his reluctant son.")
(2, 'Soorarai Pottru', '2020', 153, 9.1, '#####', '89,663', '*****', 'Nedumaaran Rajangam "Maara" sets out to make the common man fly and in the process takes on the world\'s most capital intensive industry and several enemies who stand in his way.')
(3, 'The Dark Knight', '2008', 152, 9.0, '84', '2,415,236', '$534.86M', 'When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice.')
(4, 'The Godfather: Part II', '1974', 202, 9.0, '90', '1,181,867', '$57.30M', 'The early life and